In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback

from stable_baselines3.common.noise import OrnsteinUhlenbeckActionNoise
from lib.environment.ball_following.environment import Environment
import numpy as np
import os
from datetime import datetime

In [ ]:
task_training_name = "ball_following"

In [ ]:
def create_env():
    def _init():
        return Environment()
    return _init

In [ ]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [ ]:
def get_task_models_path():
    current_datetime = datetime.now()

    year = current_datetime.year
    month = current_datetime.month
    day = current_datetime.day
    hour = current_datetime.hour
    minute = current_datetime.minute
    second = current_datetime.second

    datetime_name = f"{year}_{month}_{day}_{hour}_{minute}_{second}"

    return f"models/{task_training_name}/{datetime_name}"

In [ ]:
save_path = get_task_models_path()

create_folder_if_not_exists(save_path)

In [ ]:
num_threads = 4
model_filename_prefix = "DDPG_model"

In [ ]:
env = SubprocVecEnv([create_env() for i in range(num_threads)])

n_actions = env.action_space.shape[-1]
n_actions

In [ ]:
std_dev = 0.2
tau = 0.005
gamma = 0.99
learning_rate = 0.001
policy = "MlpPolicy"

In [ ]:
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

In [ ]:
model = DDPG(
    policy=policy,
    env=env,
    tau=tau,
    gamma=gamma,
    action_noise=action_noise,
    learning_rate=learning_rate,
    verbose=2)

In [ ]:
total_timesteps = 1000000

In [ ]:
saved_model_number = 5
save_freq = total_timesteps / (saved_model_number * num_threads)
log_interval = total_timesteps / 10

In [ ]:
checkpoint_callback = CheckpointCallback(
    save_freq=save_freq,
    save_path=save_path,
    name_prefix=model_filename_prefix)

In [ ]:
model.learn(
    total_timesteps=total_timesteps,
    log_interval=log_interval,
    callback=checkpoint_callback,
    progress_bar=True)

In [ ]:
evaluation_env = Environment()

for i in range(100):
    obs = evaluation_env.reset()
    reward = 0
    done = False

    while done is False:
        action, state = model.predict(obs)
        obs, recompensa, done, info = evaluation_env.step(action)
        evaluation_env.render()